<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/En%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np
from pandas.core.common import random_state

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor

SEED = 17

In [2]:
url='https://drive.google.com/file/d/1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_original = pd.read_csv(url)

In [3]:
df_original.head()

,Unnamed: 0,vigente,vigenteAvg3M,vencido,vencidoAvg3M,avgcontingenciaU1M,avgcontingenciaU3M,sumcontingenciaU1M,sumcontingenciaU3M,maxcontingenciaU1M,...,numerobancosprivados,numerobancospublicos,numerofinancieras,numerobancos,numerode5ult18meses,numerode2Bult18meses,numerode3ult18meses,edad,numeromesesreportados,ingreso
0,1,4282.155183,4679.335952,0.000000,0.000000,4923.124484,3918.527125,19692.497937,18439.941778,6653.069810,...,1,0,3,1,0,0,0,63.0,24,31888.466530
1,2,9793.122650,9450.927278,83970.243387,91126.581200,20809.053382,19126.815664,41618.106764,49280.496926,30881.253937,...,0,1,1,1,4,0,2,67.0,24,62341.141710
2,3,531419.701556,512057.084981,0.000000,0.000000,36813.149453,46144.419161,36813.149453,36789.292777,36813.149453,...,2,1,0,3,0,0,0,NaN,24,100599.764680
3,4,0.000000,0.000000,2266.728708,2633.111538,11533.815438,10488.338604,23067.630876,24025.952782,19044.717097,...,4,0,0,4,4,2,3,NaN,24,46102.904895
4,5,112394.057367,110856.020945,0.000000,0.000000,41131.333850,42820.024254,82262.667700,77614.227322,43944.580379,...,4,0,0,4,0,0,0,65.0,14,109419.216020


In [4]:
print (df_original.isnull().sum())

Unnamed: 0                   0
vigente                      0
vigenteAvg3M                 0
vencido                      0
vencidoAvg3M                 0
avgcontingenciaU1M           0
avgcontingenciaU3M           0
sumcontingenciaU1M           0
sumcontingenciaU3M           0
maxcontingenciaU1M           0
maxcontingenciaU3M           0
numcontingenciaU1M           0
garantiacomputable           0
peorcalificacionBCU          0
numeroinstituciones          0
numerobancosprivados         0
numerobancospublicos         0
numerofinancieras            0
numerobancos                 0
numerode5ult18meses          0
numerode2Bult18meses         0
numerode3ult18meses          0
edad                     11429
numeromesesreportados        0
ingreso                      0
dtype: int64


se remplazan los null por la media de la columna

In [5]:
edad_media = df_original['edad'].mean()
edad_media

45.02159532392986

In [6]:
def transfromar_DataFrame (df):
  df = df.copy()
  df = df.drop(df.columns[0],axis=1)
  #Remplazar nulls de la edad con la media de la columna
  df['edad'] = df['edad'].fillna(edad_media, inplace=False)

  #Transformar categóricas en numéricas
  df['peorcalificacionBCU'] = (df['peorcalificacionBCU'].
                              str.replace('2B','3').
                              str.replace('5','6').
                              str.replace('4','5').
                              str.replace('3','4').
                              str.replace('2A','2').
                              str.replace('1C','1').
                              str.replace('1A','0'))
  df['peorcalificacionBCU'] = pd.to_numeric(df['peorcalificacionBCU'], downcast='integer')
  return df

In [7]:
df = transfromar_DataFrame(df_original)


In [8]:
df.head()

,vigente,vigenteAvg3M,vencido,vencidoAvg3M,avgcontingenciaU1M,avgcontingenciaU3M,sumcontingenciaU1M,sumcontingenciaU3M,maxcontingenciaU1M,maxcontingenciaU3M,...,numerobancosprivados,numerobancospublicos,numerofinancieras,numerobancos,numerode5ult18meses,numerode2Bult18meses,numerode3ult18meses,edad,numeromesesreportados,ingreso
0,4282.155183,4679.335952,0.000000,0.000000,4923.124484,3918.527125,19692.497937,18439.941778,6653.069810,7995.580509,...,1,0,3,1,0,0,0,63.000000,24,31888.466530
1,9793.122650,9450.927278,83970.243387,91126.581200,20809.053382,19126.815664,41618.106764,49280.496926,30881.253937,32793.577625,...,0,1,1,1,4,0,2,67.000000,24,62341.141710
2,531419.701556,512057.084981,0.000000,0.000000,36813.149453,46144.419161,36813.149453,36789.292777,36813.149453,46144.419161,...,2,1,0,3,0,0,0,45.021595,24,100599.764680
3,0.000000,0.000000,2266.728708,2633.111538,11533.815438,10488.338604,23067.630876,24025.952782,19044.717097,17864.388357,...,4,0,0,4,4,2,3,45.021595,24,46102.904895
4,112394.057367,110856.020945,0.000000,0.000000,41131.333850,42820.024254,82262.667700,77614.227322,43944.580379,42820.024254,...,4,0,0,4,0,0,0,65.000000,14,109419.216020


In [9]:
y = df.pop("ingreso")
X = df.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)


In [32]:
# La linea base es el promedio de los ingresos
baseline_preds = y_train.mean()
# Baseline errors, and display average baseline error
baseline_MAPE = 100*sum(abs((baseline_preds - y_train)/y_train))/len(y_train)
print('MAPE linea base: ', round(baseline_MAPE , 2))

MAPE linea base:  58.11


In [34]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = SEED)# Train the model on training data
rf.fit(X_train,y_train);

In [36]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute percentage errors
errors = abs((predictions - y_test)/y_test)
# Print out the mean absolute percentageerror (mape)
print('MAPE:', 100*round(np.mean(errors), 2))

MAPE: 13.0


In [53]:
print('MAPE:', 100*round(np.mean(errors), 6))

MAPE: 12.859300000000001


In [49]:
model = xgb.XGBRegressor(
        objective="reg:squarederror",
        eval_metric="mae",
        n_estimators=10000,
        random_state=SEED,
        max_depth=6,
        booster="gbtree",
        learning_rate=0.01,
        subsample=0.7,
        gamma=0,
        reg_alpha=0,
        reg_lambda=1,
        importance_type="gain",
        colsample_bytree=0.9,
        colsample_bynode=0.8,
        colsample_bylevel=0.8,
        verbosity=1,
        n_jobs=1,
)

In [50]:
model.fit(X_train,y_train)

XGBRegressor(colsample_bylevel=0.8, colsample_bynode=0.8, colsample_bytree=0.9,
             eval_metric='mae', learning_rate=0.01, max_depth=6,
             n_estimators=10000, objective='reg:squarederror', random_state=17,
             subsample=0.7)

In [52]:
y_preds = model.predict(X_test)
# Calculate the absolute percentage errors
errs = abs((y_preds - y_test)/y_test)
# Print out the mean absolute percentageerror (mape)
print('MAPE:', 100*round(np.mean(errors), 6))

MAPE: 12.859300000000001


In [44]:
pipe = Pipeline([('scaler',StandardScaler()),('model',model)])

In [46]:
pipe.fit(X_train,y_train)


KeyboardInterrupt: ignored

In [ ]:
y_pred = pipe.predict(X_test)
acc_test = pipe.score(X_test, y_test)
acc_train = pipe.score(X_train, y_train)

In [ ]:
#data de test
#url='https://drive.google.com/file/d/1sMkorQLOmf2cNGRwXkQLKeciGSUpL_ez/view?usp=sharing'
#url='https://drive.google.com/uc?id=' + url.split('/')[-2]
#df_test = pd.read_csv(url)

#df_test.head()